In [13]:
import pandas as pd
import numpy as np

In [14]:
df=pd.read_csv('covid_toy.csv')

In [15]:
from sklearn.impute import SimpleImputer #for missing value
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [16]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [17]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [18]:
#here age,fever is numerical column and
#Gender and City is nominal categorical data for that we use OneHotEncoding  and cough is ordinal categorical data for that we use ordinal encoding

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [21]:
X_train

,age,gender,fever,cough,city
61,81,Female,98.0,Strong,Mumbai
74,34,Female,104.0,Strong,Delhi
58,23,Male,98.0,Strong,Mumbai
54,60,Female,99.0,Mild,Mumbai
1,27,Male,100.0,Mild,Delhi
...,...,...,...,...,...
55,81,Female,101.0,Mild,Mumbai
43,22,Female,99.0,Mild,Bangalore
84,69,Female,98.0,Strong,Mumbai
14,51,Male,104.0,Mild,Bangalore


## Without Transformer

for each column we have to sepreately perform the encoding and then at last have to join them toghether 

In [24]:
#for Handling Missing Values
#here the missing values are replaced from mean
si=SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [25]:
X_test_fever

array([[102.        ],
       [100.94117647],
       [100.94117647],
       [104.        ],
       [ 98.        ],
       [ 98.        ],
       [102.        ],
       [ 98.        ],
       [103.        ],
       [101.        ],
       [ 98.        ],
       [104.        ],
       [101.        ],
       [101.        ],
       [104.        ],
       [ 99.        ],
       [101.        ],
       [104.        ],
       [100.94117647],
       [ 98.        ]])

In [27]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])



In [32]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']]).toarray()

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']]).toarray()



In [33]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [34]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)


In [35]:
X_train_transformed

array([[ 81.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 34.        , 104.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 23.        ,  98.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 60.        ,  99.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 27.        , 100.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 13.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 24.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 64.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 66.        ,  99.        ,   1.        ,   0.        ,
          0.    

## Using Column Transformer

In [36]:
from sklearn.compose import ColumnTransformer

In [41]:
trans = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [43]:
trans.fit_transform(X_train)

array([[ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  60.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.    

In [44]:
trans.transform(X_test)

array([[102.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [100.82191781,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.82191781,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  16.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  73.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          1.    